# **구글 마운트 및 데이터, 라이브러리 호출 코드**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/WalBouss/GEM

  Cloning https://github.com/WalBouss/GEM to /tmp/pip-req-build-onmmo94r
  Running command git clone --filter=blob:none --quiet https://github.com/WalBouss/GEM /tmp/pip-req-build-onmmo94r
  Resolved https://github.com/WalBouss/GEM to commit a7d55f213619e9e85506b59e559a42bf25bb1a6e
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.9 MB/s eta 0:00:00
  Created wheel for gem_torch: filename=gem_torch-1.0-py3-none-any.whl size=11626 sha256=84616c8b886cf4c6c694f113877761eff477f6942e24062420b924e76f71649a
  Stored in directory: /tmp/pip-ephem-wheel-cache-mjiyqp_d/wheels/d0/a3/cc/01f7a5be958f6bad42122edfefcb1f6479a821076d7c66524d
Successfully built gem_torch


In [3]:
!git clone https://github.com/WalBouss/GEM.git
%cd GEM
!pip install -e .

Cloning into 'GEM'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 25 (delta 4), reused 20 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 40.71 MiB | 36.37 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/GEM
Obtaining file:///content/GEM
  Preparing metadata (setup.py) ... done
  Attempting uninstall: gem_torch
    Found existing installation: gem_torch 1.0
    Uninstalling gem_torch-1.0:
      Successfully uninstalled gem_torch-1.0
  Running setup.py develop for gem_torch


In [4]:
!pip install -r requirements.txt

In [5]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 32.8 MB/s eta 0:00:00


# **데이터 불러오기**

In [6]:
input_path = "/content/drive/MyDrive/Colab Notebooks/ade20k/ADEChallengeData2016/objectInfo150.txt"
output_path = "/content/ADE20K_object150.txt"

with open(input_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

classes = []
for line in lines:
    if line.strip() and not line.startswith("Idx"):
        name = line.strip().split("\t")[-1]
        name = name.split(",")[0].strip()
        classes.append(name)

with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n".join(classes))

print(f"저장 완료: {output_path} ({len(classes)} classes)")
print(classes[:10])


✅ 저장 완료: /content/ADE20K_object150.txt (150 classes)
['wall', 'building', 'sky', 'floor', 'tree', 'ceiling', 'road', 'bed', 'windowpane', 'grass']


In [7]:
import shutil
import os

src_dir = "/content/drive/MyDrive/Colab Notebooks/ade20k/ADEChallengeData2016"
zip_path = "/content/ade20k.zip"

shutil.make_archive("/content/ade20k", 'zip', src_dir)
print(f"압축 완료: {zip_path}")

✅ 압축 완료: /content/ade20k.zip


In [8]:
!unzip -q /content/ade20k.zip -d /content/ADEChallengeData2016


# **ADE 재구현 코드**

In [9]:
import os
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision.transforms import transforms

_torch_load = torch.load
def torch_load_wrapper(*args, **kwargs):
    kwargs["weights_only"] = False
    return _torch_load(*args, **kwargs)
torch.load = torch_load_wrapper

OPENAI_DATASET_MEAN = (0.48145466, 0.4578275, 0.40821073)
OPENAI_DATASET_STD  = (0.26862954, 0.26130258, 0.27577711)

In [10]:
# ------------------------------------------------
# ADE20K Dataset
# ------------------------------------------------
class ADE20K(Dataset):
    PALETTE_FULL = torch.tensor([
        [  0,   0,   0], [120, 120, 120], [180, 120, 120], [  6, 230, 230], [ 80,  50,  50],
        [  4, 200,   3], [120, 120,  80], [140, 140, 140], [204,   5, 255], [230, 230, 230],
        [  4, 250,   7], [224,   5, 255], [235, 255,   7], [150,   5,  61], [120, 120,  70],
        [  8, 255,  51], [255,   6,  82], [143, 255, 140], [204, 255,   4], [255,  51,   7],
        [204,  70,   3], [  0, 102, 200], [ 61, 230, 250], [255,   6,  51], [ 11, 102, 255],
        [255,   7,  71], [255,   9, 224], [  9,   7, 230], [220, 220, 220], [255,   9,  92],
        [112,   9, 255], [  8, 255, 214], [  7, 255, 224], [255, 184,   6], [ 10, 255,  71],
        [255,  41,  10], [  7, 255, 255], [224, 255,   8], [102,   8, 255], [255,  61,   6],
        [255, 194,   7], [255, 122,   8], [  0, 255,  20], [255,   8,  41], [255,   5, 153],
        [  6,  51, 255], [235,  12, 255], [160, 150,  20], [  0, 163, 255], [140, 140, 140],
        [250,  10,  15], [ 20, 255,   0], [ 31, 255,   0], [255,  31,   0], [255, 224,   0],
        [153, 255,   0], [  0,   0, 255], [255,  71,   0], [  0, 235, 255], [  0, 173, 255],
        [ 31,   0, 255], [ 11, 200, 200], [255,  82,   0], [  0, 255, 245], [  0,  61, 255],
        [  0, 255, 112], [  0, 255, 133], [255,   0,   0], [255, 163,   0], [255, 102,   0],
        [194, 255,   0], [  0, 143, 255], [ 51, 255,   0], [  0,  82, 255], [  0, 255,  41],
        [  0, 255, 173], [ 10,   0, 255], [173, 255,   0], [  0, 255, 153], [255,  92,   0],
        [255,   0, 255], [255,   0, 245], [255,   0, 102], [255, 173,   0], [255,   0,  20],
        [255, 184, 184], [  0,  31, 255], [  0, 255,  61], [  0,  71, 255], [255,   0, 204],
        [  0, 255, 194], [  0, 255,  82], [  0,  10, 255], [  0, 112, 255], [ 51,   0, 255],
        [  0, 194, 255], [  0, 122, 255], [  0, 255, 163], [255, 153,   0], [  0, 255,  10],
        [255, 112,   0], [143, 255,   0], [ 82,   0, 255], [163, 255,   0], [255, 235,   0],
        [  8, 184, 170], [133,   0, 255], [  0, 255,  92], [184,   0, 255], [255,   0,  31],
        [  0, 184, 255], [  0, 214, 255], [255,   0, 112], [ 92, 255,   0], [  0, 224, 255],
        [112, 224, 255], [ 70, 184, 160], [163,   0, 255], [153,   0, 255], [ 71, 255,   0],
        [255,   0, 163], [255, 204,   0], [255,   0, 143], [  0, 255, 235], [133, 255,   0],
        [255,   0, 235], [245,   0, 255], [255,   0, 122], [255, 245,   0], [ 10, 190, 212],
        [214, 255,   0], [  0, 204, 255], [ 20,   0, 255], [255, 255,   0], [  0, 153, 255],
        [  0,  41, 255], [  0, 255, 204], [ 41,   0, 255], [ 41, 255,   0], [173,   0, 255],
        [  0, 245, 255], [ 71,   0, 255], [122,   0, 255], [  0, 255, 184], [  0,  92, 255],
        [184, 255,   0], [  0, 133, 255], [255, 214,   0], [ 25, 194, 194], [102, 255,   0],
        [ 92,   0, 255]
    ], dtype=torch.uint8)

    def __init__(self,
                 root,
                 split='validation',
                 transform=None,
                 ignore_index=-1,
                 resize_mask=False,
                 classes_txt='/content/ADE20K_object150.txt'):
        super().__init__()
        self.root = root
        self.split = split
        self.ignore_index = ignore_index
        self.transform = transform if transform is not None else SegmentationTransforms(
            (448, 448), resize_mask=resize_mask
        )
        self.image_dir = os.path.join(root, 'images', split)
        self.label_dir = os.path.join(root, 'annotations', split)

        with open(classes_txt, 'r', encoding='utf-8') as f:
            self.CLASSES = [line.strip() for line in f]
        assert len(self.CLASSES) == 150, f"Expected 150 classes, got {len(self.CLASSES)}"

        pal_full = self.PALETTE_FULL.numpy().astype(np.uint32)
        pal = pal_full[1:] if pal_full.shape[0] == 151 else pal_full
        assert pal.shape[0] == 150, f"Palette must be 150 colors, got {pal.shape[0]}"

        self._pal_150 = pal
        packed_pal = (pal[:, 0] << 16) | (pal[:, 1] << 8) | pal[:, 2]
        self._pack_palette = {int(p): i for i, p in enumerate(packed_pal)}

        self.image_files = sorted([
            os.path.join(self.image_dir, f)
            for f in os.listdir(self.image_dir)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        self.label_files = sorted([
            os.path.join(self.label_dir, f)
            for f in os.listdir(self.label_dir)
            if f.lower().endswith('.png')
        ])
        assert len(self.image_files) == len(self.label_files), "Image/Label count mismatch"

    def __len__(self):
        return len(self.image_files)

    def _rgb_mask_to_index(self, rgb_target: torch.Tensor) -> torch.Tensor:
        rgb = rgb_target.permute(1, 2, 0).contiguous().numpy().astype(np.uint32)
        packed = (rgb[..., 0] << 16) | (rgb[..., 1] << 8) | rgb[..., 2]
        flat = packed.reshape(-1)
        mapped = np.full(flat.shape, self.ignore_index, dtype=np.int64)
        for k, v in self._pack_palette.items():
            mapped[flat == k] = v
        idx_mask = mapped.reshape(packed.shape)
        return torch.from_numpy(idx_mask).long()

    def __getitem__(self, idx):

        img_path = self.image_files[idx]
        label_path = self.label_files[idx]

        img_pil = Image.open(img_path).convert("RGB")
        lab_im = Image.open(label_path)
        lab_np = np.array(lab_im)

        if lab_np.ndim == 2:
            idx_mask = lab_np.astype(np.int64)
            bg = (idx_mask == 0)
            idx_mask = idx_mask - 1
            idx_mask[bg] = self.ignore_index
            index_mask = torch.from_numpy(idx_mask).long()
            images, _ = self.transform(img_pil, img_pil)
        else:
            images, rgb_target = self.transform(img_pil, lab_im.convert("RGB"))
            index_mask = self._rgb_mask_to_index(rgb_target)

        return images, index_mask


class ToTensorMask(nn.Module):
    def __init__(self):
        super(ToTensorMask, self).__init__()

    def forward(self, mask):
        return torch.as_tensor(np.array(mask), dtype=torch.uint8).permute(2, 0, 1)


class SegmentationTransforms(object):
    def __init__(self, size, img_transforms=None, resize_mask=False):
        self.img_transforms = img_transforms if img_transforms is not None else transforms.Compose([
            transforms.Resize(size=size, interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(OPENAI_DATASET_MEAN, OPENAI_DATASET_STD),
        ])
        self.mask_transforms = transforms.Compose([
            transforms.Resize(size=size) if resize_mask else nn.Identity(),
            ToTensorMask(),
        ])

    def __call__(self, image, mask):
        return self.img_transforms(image), self.mask_transforms(mask)

In [11]:
# ------------------------------------------------
# Zero-Shot Evaluator
# ------------------------------------------------
import torch.nn.functional as F
from torchmetrics.classification import MulticlassJaccardIndex
from einops import rearrange
from tqdm import tqdm
import gem

class ZeroShotSegmentation(torch.nn.Module):
    def __init__(self, model, tokenizer, model_name, patch_size=16, device='cpu',
                 class_chunk=32):
        super(ZeroShotSegmentation, self).__init__()

        self.model_name = model_name
        self.device = device

        self.gem_model = model
        self.gem_model.to(device)
        self.gem_model.eval()
        self.patch_size = patch_size
        self.tokenizer = tokenizer
        self.class_chunk = class_chunk

    def _get_text_embedding(self, classes: list):
        prompts = [f'a photo of a {cls}.' for cls in classes]

        tokenized_prompts = self.tokenizer(prompts).to(self.device)

        text_embedding = self.gem_model.model.encode_text(tokenized_prompts)
        text_embedding = F.normalize(text_embedding, dim=-1)
        return text_embedding.unsqueeze(0)

    def _visual_features(self, image):
        with torch.inference_mode(), torch.cuda.amp.autocast(enabled=image.is_cuda):
            feat, _ = self.gem_model.model.visual(image)
            feat = F.normalize(feat, dim=-1)[:, 1:]
        return feat

    def inference_chunked(self, image, text_embedding, mask_shape_hw):
        B, _, H, W = image.shape

        feat = self._visual_features(image)
        C = text_embedding.shape[1]

        chunks = []
        for s in range(0, C, self.class_chunk):
            te = text_embedding[:, s:s+self.class_chunk]
            lg = 100.0 * torch.matmul(feat, te.transpose(1, 2))
            chunks.append(lg)
            del te, lg
            if image.is_cuda:
                torch.cuda.empty_cache()

        logits_flat = torch.cat(chunks, dim=-1)

        h, w = H // self.patch_size, W // self.patch_size
        logits = logits_flat.view(B, h, w, C)
        logits = logits.permute(0, 3, 1, 2).contiguous()

        logits = F.interpolate(logits, size=mask_shape_hw, mode='bilinear', align_corners=False)
        pred = logits.argmax(1)
        return pred, logits, feat

    @torch.no_grad()
    def eval_dataset(self, dataloader, classes, device):
        C = len(classes)
        text_embedding = self._get_text_embedding(classes)

        confmat = np.zeros((C, C), dtype=np.int64)

        for images, masks in tqdm(dataloader):
            images = images.to(device)

            pred_gem, _, _ = self.inference_chunked(images, text_embedding, masks.shape[-2:])

            B = images.shape[0]
            for b in range(B):
                gt = masks[b].reshape(-1).cpu()
                pr = pred_gem[b].reshape(-1).cpu()

                valid = gt >= 0
                gt = gt[valid]
                pr = pr[valid]

                idx = gt * C + pr
                cm_flat = torch.bincount(idx, minlength=C*C)
                cm = cm_flat.view(C, C).numpy()
                confmat += cm

        tp = np.diag(confmat).astype(np.float64)
        fp = confmat.sum(axis=0) - tp
        fn = confmat.sum(axis=1) - tp
        denom = tp + fp + fn
        iou = np.where(denom > 0, tp / denom, np.nan)
        miou = float(np.nanmean(iou) * 100.0)

        print('\n======================================')
        print(f'✅ 최종 mIoU: {miou:.2f}  (ADE20K 150-class)')
        print('======================================')

        return miou


/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [12]:
# ------------------------------------------------
# Main
# ------------------------------------------------
def main(model_name, device, pretrained,patch_size=16, root_path_ade='', batch_size=1, classes_txt='/content/ADE20K_object150.txt'):
    dataset = ADE20K(root=root_path_ade, split='validation',
                    transform=SegmentationTransforms((448, 448), resize_mask=(batch_size > 1)),
                    ignore_index=-1,classes_txt=classes_txt)
    classes_list = list(dataset.CLASSES)
    if os.name == 'nt':
        num_workers = 0
        persistent_workers = False
    else:
        num_workers = max(2, (os.cpu_count() or 4) // 2)
        persistent_workers = True

    test_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers,
        pin_memory=(device.type == "cuda"),
        persistent_workers=persistent_workers
    )

    model = gem.create_gem_model(model_name=model_name, pretrained=pretrained)
    tokenizer = gem.get_tokenizer(model_name=model_name)

    evaluator = ZeroShotSegmentation(model=model,device=device, patch_size=patch_size,
                                     model_name=model_name,tokenizer=tokenizer,class_chunk=32)

    miou = evaluator.eval_dataset(
        dataloader=test_loader,
        classes=classes_list,
        device=device
    )
    return miou


if __name__ == '__main__':
    patch_size    = 16
    model_name    = 'ViT-B-16'
    pretrained    = 'openai'
    root_path_ade = '/content/ADEChallengeData2016'
    classes_txt   = '/content/ADE20K_object150.txt'

    print('########################################')
    print(f'model: {model_name} | pretrained: {pretrained}')
    print('########################################')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    final_miou = main(
        model_name=model_name,
        pretrained=pretrained,
        device=device,
        patch_size=patch_size,
        root_path_ade=root_path_ade,
        classes_txt=classes_txt,
        batch_size=1
    )
    print(f"Final mIoU: {final_miou:.2f}")

########################################
model: ViT-B-16 | pretrained: openai
########################################


100%|████████████████████████████████████████| 351M/351M [00:02<00:00, 139MiB/s]
  0%|          | 0/2000 [00:00<?, ?it/s]/tmp/ipython-input-3622706785.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.inference_mode(), torch.cuda.amp.autocast(enabled=image.is_cuda):
100%|██████████| 2000/2000 [00:52<00:00, 37.78it/s]



✅ 최종 mIoU: 14.72  (ADE20K 150-class)
Final mIoU: 14.72
